In [1]:
import polars as pl
import numpy as np
import pandas as pd

import matplotlib as mpl
import matplotlib.pyplot as plt

import seaborn as sns
import seaborn.objects as so

ModuleNotFoundError: No module named 'seaborn'